In [ ]:
import numpy as np
from collections import defaultdict
import math
from functools import reduce
import json

# 优化的数据处理

In [ ]:
def model_coefficients(x, coeffs):
    """
    计算多项式的输出
    """
    a, b, c, d, e, f, g, h, i = coeffs
    x1, x2, x3 = x
    return a*x1**2 + b*x2**2 + c*x3**2 + d*x1*x2 + e*x1*x3 + f*x2*x3 + g*x1 + h*x2 + i*x3

def validate(coeffs, best, others):
    """
    验证是否所有other的y值都小于best的y值
    """
    y_best = model_coefficients(best, coeffs)
    y_others = np.array([model_coefficients(other, coeffs) for other in others])
    for index,other in enumerate(y_others):
        if y_best < other:
            print(others[index])
            print('best:{}\tother:{}'.format(y_best, other))
    # 检查是否所有y_others都小于y_best
    valid = np.all(y_others <= y_best)
    
    return valid, y_best, y_others

def enhanced_scale(arr, scale_min=1, scale_max=10):
    """
    改进的归一化函数：每个维度独立标准化，再缩放到1-10之间
    """
    arr = np.array(arr)
    scaled = np.zeros_like(arr)
    
    for i in range(arr.shape[1]):
        col = arr[:, i]
        # 标准化：均值为0，方差为1
        col_standardized = (col - np.mean(col)) / np.std(col)
        # 缩放到1-10
        scaled[:, i] = scale_min + (col_standardized - np.min(col_standardized)) * (scale_max - scale_min) / (np.max(col_standardized) - np.min(col_standardized))
    
    return np.round(scaled).astype(int)  # 取整保证整数系数

In [ ]:
with open('../count_datas/coefficients.json','r') as f:
    xishu = json.load(f)
xishu

In [ ]:
res = {}
with open('../count_datas/f1_coefficient_valid.json', 'r') as f:
    tmp_datas = json.load(f)
res['f1'] = tmp_datas

In [ ]:
for key in xishu.keys():
    print(key)
    optimal_coeffs = xishu[key]
    print('optimal_coeffs:{}'.format(optimal_coeffs))
    now_res = res['f1'][key]
    best_str = now_res['best_str']
    print('best_str:{}'.format(best_str))
    other_str = now_res['other_str']
    best = best_str.split('\t')
    best = [float(item) for item in best]
    others = []
    for item in other_str.split('\n'):
        if item == '':
            continue
        tmp_list = item.split('\t')
        tmp_list = [float(_) for _ in tmp_list]
        others.append(tmp_list)
    
    # 删除best那组数据
    others = [other for other in others if not np.allclose(other, best)]
    # 合并best和others
    all_solutions = [best] + others

    
    # 归一化处理所有数据
    processed_solutions = enhanced_scale(all_solutions, scale_min=1, scale_max=10)

    processed_best = processed_solutions[0]
    processed_others = processed_solutions[1:]



    print(processed_best)
    print("Optimal coefficients:", optimal_coeffs)

    # 验证是否所有other的y值都小于best的y值
    valid, y_best, y_others = validate(optimal_coeffs, processed_best, processed_others)

    # 输出验证结果
    if valid:
        print("验证成功！所有other的y值都小于best的y值。")
    else:
        print("验证失败！以上y值不符合要求\n\n")
        # print("y_best:", y_best)
        # print("y_others:", y_others)
    


In [ ]:
for key in xishu.keys():

    print(key)
    optimal_coeffs = xishu[key]
    print('optimal_coeffs:{}'.format(optimal_coeffs))
    now_res = res['f1'][key]
    valid_best_str = now_res['best_str']
    valid_other_str = now_res['other_str']
    
    best = valid_best_str.split('\t')
    best = [float(item) for item in best]
    others = []
    for item in valid_other_str.split('\n'):
        if item == '':
            continue
        tmp_list = item.split('\t')
        tmp_list = [float(_) for _ in tmp_list]
        others.append(tmp_list)
    
    # 删除best那组数据
    # others = [other for other in others if not np.allclose(other, best)]
    # 合并best和others
    all_solutions = [best] + others

    
    # 归一化处理所有数据
    processed_solutions = enhanced_scale(all_solutions, scale_min=1, scale_max=10)

    processed_best = processed_solutions[0]
    processed_others = processed_solutions[1:]

    
    s = ''
    for other,processed_other in zip(others, processed_others):
        s += '{}\t{}\t{}\t{}\t{}\t{}\t{}'.format(other[0], other[1], other[2], processed_other[0], processed_other[1], processed_other[2],model_coefficients(processed_other, optimal_coeffs))
        # s += '{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}'.format(other[0], other[1], other[2], processed_other[0], processed_other[1], processed_other[2], optimal_coeffs[0], optimal_coeffs[1], optimal_coeffs[2], optimal_coeffs[3], optimal_coeffs[4], optimal_coeffs[5], optimal_coeffs[6], optimal_coeffs[7], optimal_coeffs[8],model_coefficients(processed_other, optimal_coeffs))
        s += '\n'

    print(key)
    print(s)


In [ ]:
processed_other[0]